In [9]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pyodbc
import pandas as pd
from google.oauth2 import service_account
from dotenv import load_dotenv
import os
from google.cloud import bigquery

## Extract Data

In [2]:
# Define the connection string to the database

conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-F43K071;DATABASE=ContosoRetailDW')


In [3]:
# Create a cursor object to interact with the database

cursor = conn.cursor()

# Execute a SQL query to retrieve the table names from the database schema

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' AND table_catalog='ContosoRetailDW'")

# Loop over the results and print each table name

for table in cursor.fetchall():
    print(table)

('DimMachine', )
('DimProductSubcategory', )
('DimProduct', )
('DimProductCategory', )
('DimEntity', )
('DimSalesTerritory', )
('DimCurrency', )
('DimOutage', )
('DimChannel', )
('DimScenario', )
('DimAccount', )
('DimPromotion', )
('DimCustomer', )
('DimEmployee', )
('DimDate', )
('FactExchangeRate', )
('FactITMachine', )
('FactITSLA', )
('FactOnlineSales', )
('FactStrategyPlan', )
('FactSales', )
('FactInventory', )
('sysdiagrams', )
('FactSalesQuota', )
('DimStore', )
('DimGeography', )


In [19]:
# Execute a SQL query to retrieve data from the desired table

sql_query = "SELECT * FROM dbo.DimMachine"
machine_df = pd.read_sql(sql_query, conn)

C:\Users\MICHELLE\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
# Print Your DataFrame

print(machine_df.head())

   MachineKey MachineLabel  StoreKey MachineType  \
0           1   POS0100101         1       POS01   
1           2   POS0100102         1       POS01   
2           3   POS0100103         1       POS01   
3           4   POS0100104         1       POS01   
4           5   POS0100105         1       POS01   

                     MachineName  \
0  Fabrikam Point of Sale 4-in-1   
1  Fabrikam Point of Sale 4-in-1   
2  Fabrikam Point of Sale 4-in-1   
3  Fabrikam Point of Sale 4-in-1   
4  Fabrikam Point of Sale 4-in-1   

                                  MachineDescription      VendorName  \
0  Kit contains one convenient box, barcode scann...  Fabrikam, Inc.   
1  Kit contains one convenient box, barcode scann...  Fabrikam, Inc.   
2  Kit contains one convenient box, barcode scann...  Fabrikam, Inc.   
3  Kit contains one convenient box, barcode scann...  Fabrikam, Inc.   
4  Kit contains one convenient box, barcode scann...  Fabrikam, Inc.   

  MachineOS MachineSource            

In [21]:
# Execute a SQL query to retrieve data from the desired table

sql_query = "SELECT * FROM dbo.DimProduct"
product_df = pd.read_sql(sql_query, conn)

C:\Users\MICHELLE\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
# Print Your DataFrame

print(product_df.head())

   ProductKey ProductLabel                          ProductName  \
0           1      0101001  Contoso 512MB MP3 Player E51 Silver   
1           2      0101002    Contoso 512MB MP3 Player E51 Blue   
2           3      0101003     Contoso 1G MP3 Player E100 White   
3           4      0101004    Contoso 2G MP3 Player E200 Silver   
4           5      0101005       Contoso 2G MP3 Player E200 Red   

                                  ProductDescription  ProductSubcategoryKey  \
0                 512MB USB driver plays MP3 and WMA                      1   
1                 512MB USB driver plays MP3 and WMA                      1   
2  1GB flash memory and USB driver plays MP3 and WMA                      1   
3   2GB flash memory, LCD display, plays MP3 and WMA                      1   
4   2GB flash memory, LCD display, plays MP3 and WMA                      1   

   Manufacturer BrandName ClassID ClassName StyleID  ... UnitCost UnitPrice  \
0  Contoso, Ltd   Contoso       1   Economy

## Load Data

In [15]:
from google.oauth2.service_account import Credentials

credentials = Credentials.from_service_account_file(credentials_path)

In [16]:
load_dotenv()
credentials_path = os.path.join(os.environ['USERPROFILE'], 'Documents', 'credentials_path', 'gbq.json')
credentials = Credentials.from_service_account_file(credentials_path)

In [17]:
machine_df.to_gbq(credentials=credentials, destination_table='etl_sqlserver.machine_df',
                 if_exists='replace',
                   project_id='second-grail-352314'
                  )

100%|██████████| 1/1 [00:00<?, ?it/s]


In [23]:
product_df.to_gbq(credentials=credentials, destination_table='etl_sqlserver.product_df',
                 if_exists='replace',
                   project_id='second-grail-352314'
                  )

100%|██████████| 1/1 [00:00<?, ?it/s]


In [24]:
### Schemas

# FactSales==DimDate, Dimchannel,DimStore,DimProduct,DimPromotion,DimCurrency
# FactExchangeRate==DimCurrency
# FactInventory==DimStore,DimDate,DimProduct,DimCurrency
# FactITMachine==DimMachine,DimDate
# FactITSLA == DimDate,DimStore,DimMachine,DimOutage
# FactOnlineSales==DimDate,DimStore,DimProduct,DimPromotion,DimCurrency,DimCustomer
# FactSalesQuota==DimSalesQuota,DimChannel,DimStore,DimProduct,DimDate,DimCurrency,DimScenario
# FactStrategyPlan==DimDate,DimScenario,DimAccount,DimCurrency,DimProductCategory